In [1]:
using DataFrames, CSV, StringEncodings, Dictionaries, Dates, StatsBase, HTTP, JSON, ProgressBars

In [187]:
yr = 2022
datadir = joinpath("..", "Data", string(yr))

"../Data/2022"

In [194]:
datadf = DataFrame(CSV.File(joinpath(datadir, "pitching_data.csv")))
resdf = DataFrame(CSV.File(joinpath(datadir, "game_results.csv")))

totaldf = leftjoin(datadf, resdf, on=:game_pk)
CSV.write(joinpath(datadir, "pitching_data_with_results.csv"), totaldf)

"../Data/2022/pitching_data_with_results.csv"

In [195]:
# Here we prepare the raw SP data frames to merge them correctly

spnames = DataFrame(CSV.File(joinpath(datadir, "SP_names.csv")))
spteams = DataFrame(CSV.File(joinpath(datadir, "SP_teams.csv")))
datadf = DataFrame(CSV.File(joinpath(datadir, "pitching_data_with_results.csv")))

spnames.join_key = string.(spnames.game_pk) .* string.(spnames.pitches) .* string.(spnames.babip) .* string.(spnames.launch_speed) .* string.(spnames.launch_angle)
spteams.join_key = string.(spteams.game_pk) .* string.(spteams.pitches) .* string.(spteams.babip) .* string.(spteams.launch_speed) .* string.(spteams.launch_angle)

a = size(spnames, 1)
b = size(spteams, 1)

println("Height of each DF: names = $a, teams = $b")

spnames = spnames[!, [:join_key, :game_pk, :player_id, :player_name]]
spteams = spteams[!, [:join_key, :player_id, :player_name]]
rename!(spteams, [:player_id, :player_name] .=> [:team_id, :team_name])

spdf = innerjoin(spnames, spteams, on=:join_key)
c = size(spdf, 1)

println("Height of joined df: $c")

if a == b && b == c
    spdf.join_key = string.(spdf.game_pk) .* string.(spdf.team_id)
    rename!(spdf, [:player_id, :player_name] .=> [:spitcher_id, :spitcher_name])
    spdf = spdf[!, [:join_key, :spitcher_id, :spitcher_name]]
    
    datadf.join_key = string.(datadf.game_pk) .* string.(datadf.player_id)
    leftjoin!(datadf, spdf, on=:join_key)
end

Height of each DF: names = 4860, teams = 4860
Height of joined df: 4860


Row,pitches,player_id,player_name,game_date,game_pk,total_pitches,pitch_percent,ba,iso,babip,slg,woba,xwoba,xba,hits,abs,launch_speed,launch_angle,spin_rate,velocity,effective_speed,whiffs,swings,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance,pitcher_run_exp,run_exp,bat_speed,swing_length,pa,bip,singles,doubles,triples,hrs,so,k_percent,bb,bb_percent,api_break_z_with_gravity,api_break_z_induced,api_break_x_arm,api_break_x_batter_in,hyper_speed,bbdist,hardhit_percent,barrels_per_bbe_percent,barrels_per_pa_percent,release_pos_z,release_pos_x,plate_x,plate_z,obp,barrels_total,batter_run_value_per_100,xobp,xslg,pitcher_run_value_per_100,xbadiff,xobpdiff,xslgdiff,wobadiff,swing_miss_percent,arm_angle,attack_angle,attack_direction,swing_path_tilt,rate_ideal_attack_angle,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,GameDate,HomeID,HomeRuns,AwayID,AwayRuns,join_key,spitcher_id,spitcher_name
,Int64,Int64,String3,Date,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Int64?,Float64,Float64?,Int64,Int64,Int64,Float64?,Float64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Float64,Float64,Missing,Missing,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Missing,Missing,Missing,Missing,Missing,Missing,Date,Int64,Int64,Int64,Int64,String,Int64?,String31?
1,237,146,MIA,2022-08-26,661355,237,100.0,0.356,0.222,0.389,0.578,0.421,0.345,0.265,16,45,89.9,21.0,2352,90.3,90.08,31,114,123,-0.2,6.33,106,151,121,145,310,320,293,-4.198,4.198,missing,missing,51,38,10,4,0,2,8,15.7,5,9.8,2.13068,0.70682,0.36996,0.02405,90.8,168,39.4737,10.5263,7.84314,5.84,-0.27,-0.01,2.23,0.412,4,1.77131,0.337,0.451,-1.77131,0.091,0.075,0.127,0.076,27.2,37.3,missing,missing,missing,missing,missing,missing,2022-08-26,146,6,119,10,661355146,667297,"Nance, Tommy"
2,235,142,MIN,2022-09-17,662958,249,94.4,0.226,0.094,0.275,0.321,0.284,0.305,0.264,12,53,77.6,-1.7,2284,88.8,88.58,29,111,124,-0.2,6.17,110,150,115,144,280,303,284,0.523,-0.523,missing,missing,65,43,9,2,0,1,12,18.5,8,12.3,2.47766,0.4615,0.0806,-0.1835,90.2,122,23.2558,2.32558,1.53846,6.02,-1.23,0.35,2.19,0.349,1,-0.222553,0.381,0.329,0.222553,-0.038,-0.032,-0.008,-0.021,26.1,45.9,missing,missing,missing,missing,missing,missing,2022-09-17,114,7,142,6,662958142,680739,"Winder, Josh"
3,228,111,BOS,2022-07-22,663222,228,100.0,0.509,0.351,0.558,0.86,0.597,0.452,0.355,29,57,90.7,14.1,2187,87.9,88.12,24,119,109,0.2,6.35,110,152,125,150,284,323,292,-13.866,13.866,missing,missing,63,48,19,5,0,5,9,14.3,5,7.9,2.49232,0.69485,0.1993,-0.01193,92.4,162,52.0833,14.5833,11.1111,5.43,0.11,0.17,2.28,0.556,7,6.08158,0.417,0.638,-6.08158,0.154,0.139,0.222,0.145,20.2,30.5,missing,missing,missing,missing,missing,missing,2022-07-22,111,5,141,28,663222111,543135,"Eovaldi, Nathan"
4,224,145,CWS,2022-06-21,661481,224,100.0,0.195,0.098,0.304,0.293,0.289,0.252,0.158,8,41,84.2,8.0,2512,90.6,90.51,28,94,130,-0.1,6.11,108,149,117,146,300,333,296,1.054,-1.054,missing,missing,51,26,6,1,0,1,17,33.3,7,13.7,2.32129,0.48505,-0.03103,-0.07799,90.6,178,26.9231,3.84615,1.96078,6.11,-1.46,0.44,2.13,0.327,1,-0.470536,0.296,0.224,0.470536,0.037,0.031,0.069,0.037,29.8,47.0,missing,missing,missing,missing,missing,missing,2022-06-21,145,7,141,6,661481145,656302,"Cease, Dylan"
5,223,113,CIN,2022-08-31,662995,223,100.0,0.182,0.182,0.154,0.364,0.297,0.357,0.256,8,44,88.0,15.1,2404,87.9,87.74,19,116,107,-0.2,6.09,108,146,115,143,288,314,285,0.922,-0.922,missing,missing,53,41,4,2,0,2,4,7.5,7,13.2,2.44054,0.52471,0.2155,-0.23928,91.1,161,36.5854,7.31707,5.66038,5.72,0.03,0.05,2.45,0.302,3,-0.413453,0.371,0.431,0.4

In [196]:
# Add on winning team

HomeTeamWin = datadf.HomeRuns .> datadf.AwayRuns
IsHomeTeam = datadf.player_id .== datadf.HomeID

datadf.winner = .!(xor.(HomeTeamWin, IsHomeTeam))

sum(ismissing.(datadf.spitcher_id))

0

In [197]:
datadf = coalesce.(datadf, NaN)

CSV.write(joinpath(datadir, "PitchingDataWithResultsAndStarters.csv"), datadf)

"../Data/2022/PitchingDataWithResultsAndStarters.csv"

In [198]:
homedf = datadf[datadf.player_id .== datadf.HomeID, :]
sort!(homedf, [:game_pk])
awaydf = datadf[datadf.player_id .== datadf.AwayID, :]
sort!(awaydf, [:game_pk])

sort!(datadf, [:game_pk])

datadf.offensive_xwoba = NaN .* zeros(size(datadf, 1))
datadf.offensive_launch_angle = NaN .* zeros(size(datadf, 1))
datadf.offensive_launch_speed = NaN .* zeros(size(datadf, 1))
datadf.offensive_hardhit_percent = NaN .* zeros(size(datadf, 1))

datadf[datadf.player_id .== datadf.HomeID, [:offensive_xwoba, 
                        :offensive_launch_angle, :offensive_launch_speed, 
                                                :offensive_hardhit_percent]] = Array(awaydf[!, [:xwoba, :launch_angle,
                                                                                    :launch_speed, :hardhit_percent]]);

datadf[datadf.player_id .== datadf.AwayID, [:offensive_xwoba, 
                        :offensive_launch_angle, :offensive_launch_speed, 
                                                :offensive_hardhit_percent]] = Array(homedf[!, [:xwoba, :launch_angle,
                                                                                    :launch_speed, :hardhit_percent]]);


In [199]:
# Here let's add on a last five game offense / last game with pitcher 

last5_o_xwoba = NaN .* ones(size(datadf, 1))
last5_o_launch_angle = NaN .* ones(size(datadf, 1))
last5_o_launch_speed = NaN .* ones(size(datadf, 1))
last5_o_hardhit_percent = NaN .* ones(size(datadf, 1))
win_pct_to_date = NaN .* ones(size(datadf, 1))

last_p_xwoba = NaN .* ones(size(datadf, 1))
last_p_launch_angle = NaN .* ones(size(datadf, 1))
last_p_launch_speed = NaN .* ones(size(datadf, 1))
last_p_hardhit_percent = NaN .* ones(size(datadf, 1))

idx = 1

for r = ProgressBar(eachrow(datadf))
    last5df = datadf[(datadf.game_date .<= r.game_date) .& 
                    (datadf.player_id .== r.player_id) .&
                    (datadf.game_pk .!= r.game_pk), :]
    sort!(last5df, :game_date)

    win_pct_to_date[idx] = mean(last5df.winner)
    
    last5df = last(last5df, 5)

    last5_o_hardhit_percent[idx] = mean(last5df.offensive_hardhit_percent)
    last5_o_launch_angle[idx] = mean(last5df.offensive_launch_angle)
    last5_o_launch_speed[idx] = mean(last5df.offensive_launch_speed)
    last5_o_xwoba[idx] = mean(last5df.offensive_xwoba)

    lastpitchdf = datadf[(datadf.game_date .<= r.game_date) .& 
                    (datadf.player_id .== r.player_id) .&
                    (datadf.game_pk .!= r.game_pk) .& 
                    (datadf.spitcher_id .== r.spitcher_id), :]

    sort!(last5df, :game_date)
    last5df = last(last5df, 1)     
    
    last_p_hardhit_percent[idx] = mean(lastpitchdf.hardhit_percent)
    last_p_launch_angle[idx] = mean(lastpitchdf.launch_angle)
    last_p_launch_speed[idx] = mean(lastpitchdf.launch_speed)
    last_p_xwoba[idx] = mean(lastpitchdf.xwoba)

    idx = idx + 1
end

datadf.last5_o_hardhit_percent = last5_o_hardhit_percent
datadf.last5_o_launch_angle = last5_o_launch_angle
datadf.last5_o_launch_speed = last5_o_launch_speed
datadf.last5_o_xwoba = last5_o_xwoba

datadf.last_p_hardhit_percent = last_p_hardhit_percent
datadf.last_p_launch_angle = last_p_launch_angle
datadf.last_p_launch_speed = last_p_launch_speed
datadf.last_p_xwoba = last_p_xwoba

datadf.win_pct_to_date = win_pct_to_date



0.0%┣                                             ┫ 0/4.9k [00:00<00:00, -0s/it]
10.8%┣████▏                                  ┫ 525/4.9k [00:00<00:00, 10.4kit/s]
21.8%┣████████▎                             ┫ 1.1k/4.9k [00:00<00:00, 10.5kit/s]
33.3%┣████████████▋                         ┫ 1.6k/4.9k [00:00<00:00, 10.7kit/s]
44.6%┣█████████████████                     ┫ 2.2k/4.9k [00:00<00:00, 10.7kit/s]
56.2%┣█████████████████████▍                ┫ 2.7k/4.9k [00:00<00:00, 10.8kit/s]
67.6%┣█████████████████████████▊            ┫ 3.3k/4.9k [00:00<00:00, 10.9kit/s]
79.3%┣██████████████████████████████▏       ┫ 3.9k/4.9k [00:00<00:00, 10.9kit/s]
90.8%┣██████████████████████████████████▌   ┫ 4.4k/4.9k [00:00<00:00, 11.0kit/s]
100.0%┣█████████████████████████████████████┫ 4.9k/4.9k [00:00<00:00, 10.9kit/s]
100.0%┣█████████████████████████████████████┫ 4.9k/4.9k [00:00<00:00, 10.9kit/s]


4860-element Vector{Float64}:
 0.5882352941176471
 0.4375
 0.4
 0.5333333333333333
 0.5625
 0.4666666666666667
 0.35714285714285715
 0.5714285714285714
 0.3333333333333333
 0.25
 ⋮
 0.5203252032520326
 0.5245901639344263
 0.5040650406504065
 0.5206611570247934
 0.5081967213114754
 0.3333333333333333
 0.625
 0.34782608695652173
 0.6086956521739131

In [200]:
CSV.write(joinpath(datadir, "dataset_with_past_performance.csv"), datadf)

"../Data/2022/dataset_with_past_performance.csv"

This part joins on the odds data now

In [201]:
dataperf = DataFrame(CSV.File(joinpath(datadir, "dataset_with_past_performance.csv")))
oddsdf = DataFrame(CSV.File(joinpath(datadir, "odds_data.csv")))

oddsdf.date = Date.(oddsdf.date, "mm/dd/yy") + Dates.Year(2000)
# dataperf.game_date = Date.(dataperf.game_date, "mm/dd/yy") + Dates.Year(2000)

teammap = DataFrame(CSV.File(joinpath(datadir, "..", "TeamNameMap.csv")))
applymap = Dictionary(teammap.concise, teammap.code)

oddsdf.HomeID = getindices(applymap, oddsdf.home)
oddsdf.AwayID = getindices(applymap, oddsdf.away)

dataperf.join_key = Dates.format.(dataperf.game_date, "yyyymmdd") .* string.(Int.(dataperf.HomeID)) .* 
                        string.(Int.(dataperf.AwayID)) .* string.(Int.(dataperf.HomeRuns)) .* string.(Int.(dataperf.AwayRuns))

oddsdf.join_key = Dates.format.(oddsdf.date, "yyyymmdd") .* string.(Int.(oddsdf.HomeID)) .*
                    string.(Int.(oddsdf.AwayID)) .* string.(Int.(oddsdf.home_score)) .* string.(Int.(oddsdf.away_score))

a = size(dataperf, 1)
b = size(oddsdf, 1)

println("Rows in dataperf: $a, Rows in oddsdf: $b")

total = leftjoin(dataperf, oddsdf[!, [:join_key, :home_opener_odds, :away_opener_odds]], on=:join_key)

c = size(total, 1)

println("Rows after join: $c")


CSV.write(joinpath(datadir, "dataset_with_odds_and_performance.csv"), total)


counts = countmap(total.join_key)

for (v, c) in counts
    if c >= 3
        println(v)
    end 
end

Rows in dataperf: 4860, Rows in oddsdf: 2739
Rows after join: 4860
20220511142117311
